In [8]:
# ===================================================================
# CELL #1: MASTER SETUP - IMPORTS AND DATA LOADING
# ===================================================================
# Run this cell once to load all necessary libraries and data sources.

import pandas as pd
from pathlib import Path

# --- 1. DEFINE PATHS (from project root) ---
print("--- Defining Paths ---")
DATA_DIR = Path('data')
sider_effects_path = DATA_DIR / 'sider_side_effects.tsv'
faers_demo_path = DATA_DIR / 'ASCII' / 'DEMO25Q1.txt'
faers_drug_path = DATA_DIR / 'ASCII' / 'DRUG25Q1.txt'
faers_reac_path = DATA_DIR / 'ASCII' / 'REAC25Q1.txt'

# --- 2. LOAD SIDER DATA ---
print("--- Loading SIDER data ---")
sider_df = pd.read_csv(sider_effects_path, sep='\t')

# --- 3. LOAD FAERS DATA ---
# Add low_memory=False to suppress the DtypeWarning you were seeing.
print("--- Loading FAERS data (DEMO, DRUG, REAC) ---")
demo_df = pd.read_csv(faers_demo_path, sep='$', low_memory=False)
drug_df = pd.read_csv(faers_drug_path, sep='$', low_memory=False)
reac_df = pd.read_csv(faers_reac_path, sep='$', low_memory=False)

# --- 4. MERGE FAERS DATA ---
print("--- Merging FAERS drug and reaction tables ---")
faers_df = pd.merge(drug_df[['primaryid', 'drugname']], reac_df[['primaryid', 'pt']], on='primaryid', how='inner')
faers_df = faers_df.rename(columns={'pt': 'reaction'}) # Rename 'pt' to be more intuitive

print("\n--- SETUP COMPLETE ---")
print("The following DataFrames are now loaded and ready:")
print(f"-> sider_df: {sider_df.shape}")
print(f"-> faers_df (merged): {faers_df.shape}")

--- Defining Paths ---
--- Loading SIDER data ---
--- Loading FAERS data (DEMO, DRUG, REAC) ---
--- Merging FAERS drug and reaction tables ---

--- SETUP COMPLETE ---
The following DataFrames are now loaded and ready:
-> sider_df: (153663, 4)
-> faers_df (merged): (28680588, 3)


In [9]:
# --- 2. Analyze Most Common Drugs and Reactions in FAERS ---
print("--- Top 20 Most Reported Drugs in this FAERS Quarter ---")
display(faers_df['drugname'].value_counts().head(20))

print("\n--- Top 20 Most Reported Reactions in this FAERS Quarter ---")
display(faers_df['reaction'].value_counts().head(20))

--- Top 20 Most Reported Drugs in this FAERS Quarter ---


drugname
METHOTREXATE                  826261
ACTEMRA                       816221
PREDNISONE                    713210
SULFASALAZINE                 682950
INFLECTRA                     556065
RITUXIMAB                     551234
FOLIC ACID                    479323
ORENCIA                       376654
CETIRIZINE HYDROCHLORIDE      375811
DESOXIMETASONE                360008
LEFLUNOMIDE                   320688
HYDROXYCHLOROQUINE            304785
ENBREL                        296521
ACETAMINOPHEN                 289337
XELJANZ                       273617
PHTHALYLSULFATHIAZOLE         273483
HYDROXYCHLOROQUINE SULFATE    263774
REMICADE                      248126
HUMIRA                        236191
INFLIXIMAB                    225687
Name: count, dtype: int64


--- Top 20 Most Reported Reactions in this FAERS Quarter ---


reaction
Off label use                           485487
Fatigue                                 320398
Drug ineffective                        299551
Headache                                273294
Nausea                                  268077
Pain                                    265779
Condition aggravated                    245591
Diarrhoea                               221515
Dyspnoea                                218373
Arthralgia                              214127
Rash                                    212299
Vomiting                                207063
Malaise                                 190405
Pneumonia                               183353
Pyrexia                                 182739
Hypertension                            174119
Intentional product use issue           172925
Product use in unapproved indication    171113
Pain in extremity                       168309
Abdominal discomfort                    163935
Name: count, dtype: int64

In [10]:
# --- 3. Filter FAERS Reactions Using SIDER as a Ground Truth ---
print(f"Creating a 'ground truth' set from {len(sider_df['side_effect_name'].unique())} unique SIDER terms...")
known_side_effects = set(sider_df['side_effect_name'].str.lower())

print(f"Original FAERS data shape: {faers_df.shape}")
faers_cleaned_df = faers_df[faers_df['reaction'].str.lower().isin(known_side_effects)].copy()
print(f"Cleaned FAERS data shape: {faers_cleaned_df.shape}")

print("\n--- Top 20 Reactions after SIDER Filtering ---")
display(faers_cleaned_df['reaction'].value_counts().head(20))

Creating a 'ground truth' set from 5734 unique SIDER terms...
Original FAERS data shape: (28680588, 3)
Cleaned FAERS data shape: (22222741, 3)

--- Top 20 Reactions after SIDER Filtering ---


reaction
Fatigue                 320398
Drug ineffective        299551
Headache                273294
Nausea                  268077
Pain                    265779
Condition aggravated    245591
Diarrhoea               221515
Dyspnoea                218373
Arthralgia              214127
Rash                    212299
Vomiting                207063
Malaise                 190405
Pneumonia               183353
Hypertension            174119
Pain in extremity       168309
Abdominal discomfort    163935
Weight increased        162552
Nasopharyngitis         162038
Dizziness               162037
Asthenia                161568
Name: count, dtype: int64

In [11]:
# --- 4. Reaction Cleaning via Manual Blocklist ---
reaction_blocklist = [
    'off label use',
    'drug ineffective',
    'condition aggravated',
    'product use in unapproved indication',
    'intentional product use issue',
    'disease progression',
    'malignant neoplasm progression'
]
print(f"Shape before blocklist filtering: {faers_cleaned_df.shape}")
faers_cleaned_df = faers_cleaned_df[~faers_cleaned_df['reaction'].str.lower().isin(reaction_blocklist)].copy()
print(f"Shape after blocklist filtering: {faers_cleaned_df.shape}")

Shape before blocklist filtering: (22222741, 3)
Shape after blocklist filtering: (21647695, 3)


In [12]:
# --- 5. Build Synonym Dictionary from Existing SIDER Data ---
print("--- Building a foundational synonym dictionary from our existing SIDER files ---")

# --- Load Indications Data ---
indications_path = DATA_DIR / 'sider_indications.tsv'
indications_df = pd.read_csv(indications_path, sep='\t', header=None)
indications_df = indications_df.rename(columns={0: 'drugbank_id', 1: 'indication_name'})

# --- Combine All Known Names ---
sider_names = sider_df[['drugbank_id', 'drugbank_name']].rename(columns={'drugbank_name': 'name'})
indications_names = indications_df[['drugbank_id', 'indication_name']].rename(columns={'indication_name': 'name'})
master_mapping_df = pd.concat([sider_names, indications_names]).drop_duplicates().dropna()

# --- Build the Dictionary ---
drug_synonyms = dict(zip(master_mapping_df['name'].str.upper(), master_mapping_df['drugbank_id']))
print(f"Successfully built synonym dictionary with {len(drug_synonyms)} entries.")

# --- Verification Test ---
print("\n--- Testing the dictionary ---")
print(f"Mapping for 'SULFASALAZINE': {drug_synonyms.get('SULFASALAZINE')}")
print(f"Mapping for 'METHOTREXATE': {drug_synonyms.get('METHOTREXATE')}")
print(f"Mapping for 'ACETAMINOPHEN': {drug_synonyms.get('ACETAMINOPHEN')}")

--- Building a foundational synonym dictionary from our existing SIDER files ---
Successfully built synonym dictionary with 1249 entries.

--- Testing the dictionary ---
Mapping for 'SULFASALAZINE': None
Mapping for 'METHOTREXATE': DB00563
Mapping for 'ACETAMINOPHEN': DB00316


In [13]:
# --- 6. Apply Synonym Dictionary to Standardize Drug Names ---

print(f"Applying synonym map to {len(faers_cleaned_df)} rows...")

# Use the .map() function to create a new 'drugbank_id' column.
# We map the uppercase drugname to our dictionary.
faers_cleaned_df['drugbank_id'] = faers_cleaned_df['drugname'].str.upper().map(drug_synonyms)

# --- Verification Step ---
mapped_count = faers_cleaned_df['drugbank_id'].notna().sum()
total_count = len(faers_cleaned_df)
mapping_pct = (mapped_count / total_count) * 100
print(f"Successfully mapped {mapped_count} of {total_count} entries ({mapping_pct:.2f}%).")

# For our final dataset, we'll drop the rows we couldn't map to a standard ID.
faers_standardized_df = faers_cleaned_df.dropna(subset=['drugbank_id']).copy()
print(f"Shape of DataFrame after dropping unmapped drugs: {faers_standardized_df.shape}")

display(faers_standardized_df.head())

Applying synonym map to 21647695 rows...
Successfully mapped 6688695 of 21647695 entries (30.90%).
Shape of DataFrame after dropping unmapped drugs: (6688695, 4)


,primaryid,drugname,reaction,drugbank_id
0,100294532,LETROZOLE,Asthenia,DB01006
2,100294532,LETROZOLE,Palmar-plantar erythrodysaesthesia syndrome,DB01006
3,100294532,LETROZOLE,Metastases to liver,DB01006
6,100294532,LAPATINIB,Asthenia,DB01259
8,100294532,LAPATINIB,Palmar-plantar erythrodysaesthesia syndrome,DB01259


In [14]:
# --- 7. Final Cleaning: Remove Reactions That Are Also Indications ---

# We already built the drug_to_indications dictionary in a previous cell,
# but we will re-create it here for clarity and self-containment of this step.
print("--- Building Drug-Indication Dictionary ---")
indications_path = DATA_DIR / 'sider_indications.tsv'
indications_df = pd.read_csv(indications_path, sep='\t', header=None, names=['drugbank_id', 'indication_name'])
drug_to_indications = indications_df.groupby('drugbank_id')['indication_name'].apply(lambda x: set(x.str.lower())).to_dict()

# --- Define the Filtering Function ---
def is_likely_treatment_failure(row):
    indications = drug_to_indications.get(row['drugbank_id'], set())
    return row['reaction'].lower() in indications

print("\n--- Applying Indication Filter ---")
print(f"Shape before indication filtering: {faers_standardized_df.shape}")

# Apply the function to get a boolean Series
is_failure = faers_standardized_df.apply(is_likely_treatment_failure, axis=1)

# Keep only the rows where the reaction is NOT a likely treatment failure
final_df = faers_standardized_df[~is_failure].copy()
print(f"Shape after indication filtering: {final_df.shape}")

--- Building Drug-Indication Dictionary ---

--- Applying Indication Filter ---
Shape before indication filtering: (6688695, 4)
Shape after indication filtering: (6688695, 4)


In [15]:
# --- 8. Create Final Edge List for the Knowledge Graph ---

print("--- Creating the final drug-reaction edge list ---")
# We only need the standardized drug ID and the reaction name.
# We drop duplicates to get a unique set of relationships found in the data.
edge_list_df = final_df[['drugbank_id', 'reaction']].drop_duplicates().reset_index(drop=True)

print(f"Final edge list created with {len(edge_list_df)} unique drug-ADR relationships.")
print("This DataFrame is now ready for graph construction.")
display(edge_list_df.head())

--- Creating the final drug-reaction edge list ---
Final edge list created with 318666 unique drug-ADR relationships.
This DataFrame is now ready for graph construction.


,drugbank_id,reaction
0,DB01006,Asthenia
1,DB01006,Palmar-plantar erythrodysaesthesia syndrome
2,DB01006,Metastases to liver
3,DB01259,Asthenia
4,DB01259,Palmar-plantar erythrodysaesthesia syndrome


In [16]:
# --- 9. Save Final Edge List to File ---

print("--- Saving the final edge list to a CSV file... ---")
output_path = DATA_DIR / 'drug_adr_edge_list.csv'

edge_list_df.to_csv(output_path, index=False)

print(f"Successfully saved to: {output_path}")

--- Saving the final edge list to a CSV file... ---
Successfully saved to: data\drug_adr_edge_list.csv
